In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.functions import col, expr
import dlt
from delta.tables import DeltaTable
from pyspark.sql import SparkSession
from pyspark.sql.functions import xxhash64

@dlt.table(
    name="silver_table",
    comment="Streaming silver table from bronze stage with transformations and SCD2 columns"
)
def silver_incoming():
    bronze_df = dlt.read_stream("bronze_stage")
    config_df = dlt.read("`unity-veersa`.config_tables.config_stage_to_silver")
    active_cols = config_df.filter(F.col("is_active") == True).collect()

    transformed_df = bronze_df
    for col_meta in active_cols:
        col_name = col_meta["column_name"]
        alias_name = col_meta["alias_name"] or col_name
        if col_name in bronze_df.columns:
            col_expr = F.col(col_name)
            if col_meta["apply_trim"]:
                col_expr = F.trim(col_expr)
            if col_meta["apply_upper"]:
                col_expr = F.upper(col_expr)
            transformed_df = transformed_df.withColumn(alias_name, col_expr)

    transformed_df = transformed_df.withColumn("ingest_ts", F.current_timestamp()) \
                                   .withColumn("record_source", F.lit("bronze_stage")) \
                                   .withColumn("rowhash",xxhash64(*transformed_df.columns))

    if "city" in transformed_df.columns:
        transformed_df = transformed_df.withColumn("city", F.initcap(F.col("city")))
    if "zip_prefix" in transformed_df.columns:
        transformed_df = transformed_df.withColumn("zip_prefix", F.lpad(F.col("zip_prefix").cast("string"), 5, "0"))
    if "customer_id" in transformed_df.columns:
        transformed_df = transformed_df.drop("customer_id")
    

    return transformed_df

